# Comprehensive SessionManager Demo

This notebook demonstrates comprehensive Neptune Analytics instance management using SessionManager with automatic cleanup.

This notebook covers:
1. Start/stop instances
2. Reset instance
3. Export an instance to S3
4. Create a snapshot from an instance
5. Create multiple instances in parallel (empty, from S3, from snapshot)
6. Automatic cleanup using context manager (__exit__)

## Setup

Import the necessary libraries and set up logging.

In [1]:
import logging
import os
import asyncio

import dotenv
dotenv.load_dotenv()

from nx_neptune.session_manager import SessionManager
from nx_neptune import instance_management
import networkx as nx

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Environment Variables

Set up required environment variables for S3 operations.

In [2]:
# Required environment variables
s3_import_bucket = os.getenv('NETWORKX_S3_IMPORT_BUCKET_PATH')
s3_export_bucket = os.getenv('NETWORKX_S3_EXPORT_BUCKET_PATH')

print(f"S3 Import Bucket: {s3_import_bucket}")
print(f"S3 Export Bucket: {s3_export_bucket}")

if not all([s3_import_bucket, s3_export_bucket]):
    print("⚠️ Warning: Some environment variables are not set. S3 operations may fail.")

S3 Import Bucket: s3://nx-cit-patents/csv-import/
S3 Export Bucket: s3://nx-cit-patents/csv-export/


## Scenario 1: Start/Stop Instances

Create an instance, stop it, then start it again using SessionManager.

In [3]:
from nx_neptune.session_manager import CleanupTask

print("=== Scenario 1: Start/Stop Instances ===")

with SessionManager("start-stop-demo", cleanup_task=CleanupTask.DESTROY) as start_stop_session:
    # Create a new instance
    print("Creating new instance...")
    graph = await start_stop_session.get_or_create_graph()
    graph_name = graph['name']
    print(f"Created instance: {graph_name}")
    
    # Stop the instance
    print(f"Stopping instance...{graph_name}")
    await start_stop_session.stop_graph(graph_name)
    print("Instance stopped")
    
    # Start the instance again
    print(f"Starting instance...{graph_name}")
    await start_stop_session.start_graph(graph_name)
    print("Instance started")
    
    # List graphs to verify status
    graphs = start_stop_session.list_graphs()
    print(f"Current graphs: {len(graphs)}")
    for graph in graphs:
        print(f"  - {graph['name']}: {graph['status']}")

print("✅ Scenario 1 completed - instances automatically destroyed")

INFO - Found credentials in environment variables.


=== Scenario 1: Start/Stop Instances ===
Creating new instance...


INFO - Creating new graph named with prefix: start-stop-demo-2
INFO - [2025-12-11 13:41:33] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:42:03] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:42:34] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:43:04] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:43:34] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:44:04] Task [g-lol996x7f2] Current status: CREATING
INFO - [2025-12-11 13:44:34] Task [g-lol996x7f2] Current status: AVAILABLE
INFO - Task [g-lol996x7f2] completed at [2025-12-11 13:44:34]


Created instance: start-stop-demo-2-cdf92112-19e0-4351-847d-135f369fd9f6
Stopping instance...start-stop-demo-2-cdf92112-19e0-4351-847d-135f369fd9f6


INFO - [2025-12-11 13:44:36] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:45:06] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:45:36] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:46:06] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:46:36] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:47:06] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:47:37] Task [g-lol996x7f2] Current status: STOPPING
INFO - [2025-12-11 13:48:07] Task [g-lol996x7f2] Current status: STOPPED
INFO - Task [g-lol996x7f2] completed at [2025-12-11 13:48:07]


Instance stopped
Starting instance...start-stop-demo-2-cdf92112-19e0-4351-847d-135f369fd9f6


INFO - [2025-12-11 13:48:08] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:48:38] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:49:09] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:49:39] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:50:09] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:50:39] Task [g-lol996x7f2] Current status: STARTING
INFO - [2025-12-11 13:51:09] Task [g-lol996x7f2] Current status: AVAILABLE
INFO - Task [g-lol996x7f2] completed at [2025-12-11 13:51:09]


Instance started
Current graphs: 1
  - start-stop-demo-2-cdf92112-19e0-4351-847d-135f369fd9f6: AVAILABLE
✅ Scenario 1 completed - instances automatically destroyed


INFO - [2025-12-11 13:51:11] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:51:41] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:52:11] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:52:41] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:53:11] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:53:41] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:54:11] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:54:42] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:55:12] Task [g-lol996x7f2] Current status: DELETING
INFO - [2025-12-11 13:55:42] Task [g-lol996x7f2] Current status: DELETED
INFO - Task [g-lol996x7f2] completed at [2025-12-11 13:55:42]


## Scenario 2: Reset Instance

Create an instance, add some data, then reset it.

In [5]:
print("=== Scenario 2: Reset Instance ===")

with SessionManager("reset-demo", cleanup_task=CleanupTask.DESTROY) as reset_session:
    # Create a new instance
    print("Creating new instance...")
    graph = await reset_session.get_or_create_graph()
    graph_name = graph['name']
    print(f"Created instance: {graph_name}")
    
    # Reset the instance
    print(f"Resetting instance...{graph_name}")
    await reset_session.reset_graph(graph_name)
    print("Instance reset completed")

print("✅ Scenario 2 completed - instances automatically destroyed")

=== Scenario 2: Reset Instance ===
Creating new instance...


INFO - Creating new graph named with prefix: reset-demo
INFO - [2025-12-11 13:59:08] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 13:59:38] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 14:00:08] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 14:00:38] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 14:01:08] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 14:01:38] Task [g-o37p7zhj3a] Current status: CREATING
INFO - [2025-12-11 14:02:08] Task [g-o37p7zhj3a] Current status: AVAILABLE
INFO - Task [g-o37p7zhj3a] completed at [2025-12-11 14:02:08]
INFO - Perform reset_graph action on graph: [g-o37p7zhj3a] with skip_snapshot: [True]


Created instance: reset-demo-f069748f-c68b-4436-b50d-d3365e5fc7ee
Resetting instance...reset-demo-f069748f-c68b-4436-b50d-d3365e5fc7ee


INFO - [2025-12-11 14:02:09] Task [g-o37p7zhj3a] Current status: RESETTING
INFO - [2025-12-11 14:02:39] Task [g-o37p7zhj3a] Current status: RESETTING
INFO - [2025-12-11 14:03:10] Task [g-o37p7zhj3a] Current status: RESETTING
INFO - [2025-12-11 14:03:40] Task [g-o37p7zhj3a] Current status: AVAILABLE
INFO - Task [g-o37p7zhj3a] completed at [2025-12-11 14:03:40]


Instance reset completed
✅ Scenario 2 completed - instances automatically destroyed


INFO - [2025-12-11 14:03:41] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:04:11] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:04:41] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:05:12] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:05:42] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:06:12] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:06:42] Task [g-o37p7zhj3a] Current status: DELETING
INFO - [2025-12-11 14:07:12] Task [g-o37p7zhj3a] Current status: DELETING


## Scenario 3: Export Instance to S3

Create an instance with data and export it to S3.

In [ ]:
print("=== Scenario 3: Export Instance to S3 ===")

if s3_export_bucket:
    with SessionManager("export-demo", cleanup_task=CleanupTask.DESTROY) as export_session:
        # TODO - add export to CSV calls to session manager
        # TODO - add import from CSV calls to session manager
        pass
else:
    print("⚠️ Skipping S3 export - NETWORKX_S3_EXPORT_BUCKET_PATH not set")

print("✅ Scenario 3 completed - instances automatically destroyed")

## Scenario 4: Create Snapshot from Instance

Create an instance, add data, create a snapshot, then use the snapshot.

In [ ]:
print("=== Scenario 4: Create Snapshot from Instance ===")

with SessionManager("snapshot-demo", cleanup_task=CleanupTask.DESTROY) as snapshot_session:
    # TODO - add create snapshot calls to session manager
    # TODO - add create instance from snapshot calls to session manager
    # TODO - add destroy snapshot calls to session manager
    pass

print("✅ Scenario 4 completed - instances automatically destroyed")

## Scenario 5: Create Multiple Instances in Parallel

Create three instances in parallel: empty, from S3, and from snapshot.

In [ ]:
print("=== Scenario 5: Create Multiple Instances in Parallel ===")

with SessionManager("parallel-demo", cleanup_task=CleanupTask.DESTROY) as parallel_session:
    # First, create a source instance and snapshot for the third parallel creation
    print("Preparing snapshot for parallel creation...")
    graph = await parallel_session.get_or_create_graph()
    graph_name = graph['name']
    prep_graph_id = graph['id']
    print(f"Created instance: {graph_name}")
    
    # Create snapshot
    snapshot_id = await instance_management.create_graph_snapshot(prep_graph_id)
    print(f"Created snapshot: {snapshot_id}")
    
    # Clean up prep instance
    await instance_management.delete_na_instance(prep_graph_id)
    
    # Now create three instances in parallel
    print("Creating three instances in parallel...")
    
    async def create_empty_instance():
        graph_id = await instance_management.create_na_instance()
        print(f"Empty instance created: {graph_id}")
        return graph_id
    
    async def create_s3_instance():
        if s3_import_bucket:
            graph_id, task_id = await instance_management.create_na_instance_with_s3_import(s3_import_bucket)
            print(f"S3 instance created: {graph_id} (task: {task_id})")
            return graph_id
        else:
            print("⚠️ Skipping S3 instance - NETWORKX_S3_IMPORT_BUCKET_PATH not set")
            return None
    
    async def create_snapshot_instance():
        graph_id = await instance_management.create_na_instance_from_snapshot(snapshot_id)
        print(f"Snapshot instance created: {graph_id}")
        return graph_id
    
    # Create all instances in parallel
    results = await asyncio.gather(
        create_empty_instance(),
        create_s3_instance(),
        create_snapshot_instance(),
        return_exceptions=True
    )
    
    print(f"Parallel creation completed. Created {len([r for r in results if r and not isinstance(r, Exception)])} instances")
    
    # List all graphs
    graphs = parallel_session.list_graphs()
    print(f"Total graphs in session: {len(graphs)}")
    for graph in graphs:
        print(f"  - {graph['name']}: {graph['status']}")
    
    # Clean up snapshot
    print("Deleting snapshot...")
    await instance_management.delete_graph_snapshot(snapshot_id)
    print("Snapshot deleted")

print("✅ Scenario 5 completed - all instances automatically destroyed")

## Scenario 6: Automatic Cleanup Demonstration

Show how the context manager automatically cleans up resources.

In [ ]:
print("=== Scenario 6: Automatic Cleanup Demonstration ===")

# Create a session manager without context manager to show manual cleanup
cleanup_session = SessionManager("cleanup-demo", cleanup_task=CleanupTask.DESTROY)

try:
    # Create some instances
    print("Creating instances without context manager...")
    graph_id1, graph_id2 = await cleanup_session.create_multiple_instances(2)

    graphs = cleanup_session.list_graphs()
    print(f"Created {len(graphs)} instances")
    
    # Manually trigger cleanup using __exit__
    print("Manually triggering cleanup...")
    cleanup_session.__exit__(None, None, None)
    
    print("Manual cleanup completed")
    
except Exception as e:
    print(f"Error: {e}")
    # Ensure cleanup even on error
    cleanup_session.__exit__(type(e), e, e.__traceback__)

print("\n=== Using Context Manager for Automatic Cleanup ===")

# Demonstrate automatic cleanup with context manager
with SessionManager("auto-cleanup-demo", cleanup_task=CleanupTask.DESTROY) as auto_cleanup_session:
    print("Creating instances with context manager...")
    graph_id1, graph_id2 = await auto_cleanup_session.create_multiple_instances(2)

    graphs = auto_cleanup_session.list_graphs()
    print(f"Created {len(graphs)} instances")
    print("Exiting context manager - automatic cleanup will occur...")

print("✅ Scenario 6 completed - automatic cleanup demonstration finished")

## Summary

This notebook demonstrated comprehensive Neptune Analytics instance management using SessionManager:

1. **Start/Stop Instances**: Created, stopped, and restarted instances
2. **Reset Instance**: Added data and reset an instance to empty state
3. **Export to S3**: Exported graph data to S3 bucket
4. **Snapshot Operations**: Created snapshots and instances from snapshots
5. **Parallel Creation**: Created multiple instances simultaneously
6. **Automatic Cleanup**: Used context manager for automatic resource cleanup

Key benefits of using SessionManager:
- **Automatic Resource Management**: Context manager ensures cleanup
- **Session Isolation**: Session names provide resource grouping
- **Simplified API**: High-level operations for common tasks
- **Error Safety**: Resources cleaned up even on exceptions

All instances created during these scenarios were automatically destroyed when exiting the context manager, ensuring no resources were left running.